In [68]:
from warnings import filterwarnings
filterwarnings('ignore')

In [69]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22750 entries, 0 to 22749
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           22750 non-null  object 
 1   Date of Joining       22750 non-null  object 
 2   Gender                22750 non-null  object 
 3   Company Type          22750 non-null  object 
 4   WFH Setup Available   22750 non-null  object 
 5   Designation           22750 non-null  float64
 6   Resource Allocation   21369 non-null  float64
 7   Mental Fatigue Score  20633 non-null  float64
 8   Burn Rate             21626 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.6+ MB


In [71]:
df.describe()

,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
count,22750.000000,21369.000000,20633.000000,21626.000000
mean,2.178725,4.481398,5.728188,0.452005
std,1.135145,2.047211,1.920839,0.198226
min,0.000000,1.000000,0.000000,0.000000
25%,1.000000,3.000000,4.600000,0.310000
50%,2.000000,4.000000,5.900000,0.450000
75%,3.000000,6.000000,7.100000,0.590000
max,5.000000,10.000000,10.000000,1.000000


In [72]:
m=df.isna().sum()
m[m>0]

Resource Allocation     1381
Mental Fatigue Score    2117
Burn Rate               1124
dtype: int64

In [73]:
df.dropna(subset=['Burn Rate'], inplace=True)

In [74]:
df.isna().sum()

Employee ID                0
Date of Joining            0
Gender                     0
Company Type               0
WFH Setup Available        0
Designation                0
Resource Allocation     1278
Mental Fatigue Score    1945
Burn Rate                  0
dtype: int64

In [75]:
df.duplicated().sum()

np.int64(0)

In [76]:
df['Date of Joining']=pd.to_datetime(df['Date of Joining'])

In [77]:
def get_year_month(df):
    df['Joining_year']=df['Date of Joining'].dt.year
    df['Joining_month']=df['Date of Joining'].dt.month
    df.drop(columns=['Date of Joining'],inplace=True)
    return df

In [78]:
df=get_year_month(df)

In [79]:
df.head()

,Employee ID,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate,Joining_year,Joining_month
0,fffe32003000360033003200,Female,Service,No,2.0,3.0,3.8,0.16,2008,9
1,fffe3700360033003500,Male,Service,Yes,1.0,2.0,5.0,0.36,2008,11
2,fffe31003300320037003900,Female,Product,Yes,2.0,NaN,5.8,0.49,2008,3
3,fffe32003400380032003900,Male,Service,Yes,1.0,1.0,2.6,0.20,2008,11
4,fffe31003900340031003600,Female,Service,No,3.0,7.0,6.9,0.52,2008,7


In [80]:
x=df.drop(columns=['Employee ID', 'Burn Rate'])
y=df[['Burn Rate']]

In [81]:
x.head()

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Joining_year,Joining_month
0,Female,Service,No,2.0,3.0,3.8,2008,9
1,Male,Service,Yes,1.0,2.0,5.0,2008,11
2,Female,Product,Yes,2.0,NaN,5.8,2008,3
3,Male,Service,Yes,1.0,1.0,2.6,2008,11
4,Female,Service,No,3.0,7.0,6.9,2008,7


In [82]:
y.head()

,Burn Rate
0,0.16
1,0.36
2,0.49
3,0.20
4,0.52


In [83]:
cat=list(x.columns[x.dtypes=='object'])
con=list(x.columns[x.dtypes!='object'])

In [84]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [85]:
con_pipe=Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),
                         ('scaler', StandardScaler())])

In [86]:
cat_pipe=Pipeline(steps=[('impute', SimpleImputer(strategy='constant')),
                         ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [87]:
pre = ColumnTransformer([('con', con_pipe, con),
                           ('cat', cat_pipe, cat)]).set_output(transform='pandas')

In [88]:
x_pre=pre.fit_transform(x)
x_pre.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,con__Joining_year,con__Joining_month,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
0,-0.157448,-0.746890,-1.053224,0.0,0.723833,1.0,0.0,0.0,1.0,1.0,0.0
1,-1.038193,-1.250242,-0.398319,0.0,1.304995,0.0,1.0,0.0,1.0,0.0,1.0
2,-0.157448,0.000000,0.038284,0.0,-1.019654,1.0,0.0,1.0,0.0,0.0,1.0
3,-1.038193,-1.753594,-1.708129,0.0,1.304995,0.0,1.0,0.0,1.0,0.0,1.0
4,0.723298,1.266519,0.638613,0.0,0.142670,1.0,0.0,0.0,1.0,1.0,0.0


In [89]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x_pre, y, test_size=0.2, random_state=21)

In [90]:
xtrain.shape

(17300, 11)

In [91]:
xtest.shape

(4326, 11)

In [92]:
xtrain.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,con__Joining_year,con__Joining_month,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
22406,-1.918938,-1.753594,-1.762704,0.0,-0.438492,1.0,0.0,0.0,1.0,1.0,0.0
7389,-0.157448,-0.243538,-0.562045,0.0,0.723833,0.0,1.0,1.0,0.0,1.0,0.0
4516,-1.038193,-0.746890,-1.708129,0.0,-1.019654,1.0,0.0,0.0,1.0,0.0,1.0
14650,1.604043,0.763167,0.638613,0.0,0.433252,1.0,0.0,1.0,0.0,1.0,0.0
16088,-0.157448,-0.243538,0.256586,0.0,-0.438492,1.0,0.0,1.0,0.0,0.0,1.0


In [93]:
ytrain.head()

,Burn Rate
22406,0.08
7389,0.31
4516,0.15
14650,0.64
16088,0.56


In [94]:
xtest.head()

,con__Designation,con__Resource Allocation,con__Mental Fatigue Score,con__Joining_year,con__Joining_month,cat__Gender_Female,cat__Gender_Male,cat__Company Type_Product,cat__Company Type_Service,cat__WFH Setup Available_No,cat__WFH Setup Available_Yes
11737,-0.157448,-0.243538,0.584038,0.0,-1.310235,1.0,0.0,0.0,1.0,0.0,1.0
19222,-0.157448,0.259815,0.474887,0.0,1.595576,0.0,1.0,1.0,0.0,0.0,1.0
4948,0.723298,0.259815,-0.507470,0.0,-1.600816,0.0,1.0,1.0,0.0,0.0,1.0
1451,1.604043,1.266519,0.584038,0.0,-1.310235,1.0,0.0,0.0,1.0,1.0,0.0
11402,0.723298,-0.243538,0.092859,0.0,-0.438492,1.0,0.0,1.0,0.0,0.0,1.0


In [95]:
ytest.head()

,Burn Rate
11737,0.53
19222,0.47
4948,0.39
1451,0.57
11402,0.47


In [109]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xtrain, ytrain)

LinearRegression()

In [110]:
lr.score(xtrain, ytrain)

0.8715165825483802

In [111]:
lr.score(xtest, ytest)

0.8687652865162313

In [112]:
ypred_train = lr.predict(xtrain)
ypred_test = lr.predict(xtest)

In [113]:
ypred_train[0:5]

array([[0.1145025 ],
       [0.37909151],
       [0.17462167],
       [0.59942887],
       [0.45514409]])

In [114]:
ytrain.head()

,Burn Rate
22406,0.08
7389,0.31
4516,0.15
14650,0.64
16088,0.56


In [115]:
ypred_test[0:5]

array([[0.49580616],
       [0.51930378],
       [0.40852435],
       [0.6249166 ],
       [0.44356486]])

In [116]:
ytest.head()

,Burn Rate
11737,0.53
19222,0.47
4948,0.39
1451,0.57
11402,0.47


In [117]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def evaluate_model(model, x, y):
    ypred = model.predict(x)
    mse = mean_squared_error(y, ypred)
    rmse = mse**(1/2)
    mae = mean_absolute_error(y, ypred)
    r2 = r2_score(y, ypred)
    print(f'Mean Squared Error : {mse:.2f}')
    print(f'Root Mean Squared Error : {rmse:.2f}')
    print(f'Mean Absolute Error : {mae:.2f}')
    print(f'R2 Score : {r2:.4f}')

In [118]:
evaluate_model(model, xtrain, ytrain)

Mean Squared Error : 0.01
Root Mean Squared Error : 0.07
Mean Absolute Error : 0.05
R2 Score : 0.8715


In [119]:
evaluate_model(model, xtest, ytest)

Mean Squared Error : 0.01
Root Mean Squared Error : 0.07
Mean Absolute Error : 0.05
R2 Score : 0.8688


In [120]:
xnew = pd.read_csv('test.csv')
xnew.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
0,fffe31003300390039003000,2008-12-10,Female,Service,No,2.0,5.0,7.7
1,fffe31003300310037003800,2008-08-14,Female,Product,Yes,1.0,2.0,5.2
2,fffe33003400380035003900,2008-11-13,Male,Product,Yes,1.0,3.0,5.9
3,fffe3100370039003200,2008-02-07,Female,Service,No,3.0,6.0,4.6
4,fffe32003600390036003700,2008-07-17,Female,Product,No,2.0,5.0,6.4


In [121]:
pre

ColumnTransformer(transformers=[('con',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Designation', 'Resource Allocation',
                                  'Mental Fatigue Score', 'Joining_year',
                                  'Joining_month']),
                                ('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Gender', 'Company Type',
                                  'WFH Setup Available'])])

In [122]:
xnew_pre = pre.transform(xnew)
xnew_pre.head()

ValueError: columns are missing: {'Joining_year', 'Joining_month'}